In [12]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchinfo import summary
import torch.optim as optim
import datetime
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from metrics import deviation_metric

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
df = pd.read_csv('./Data/train_parse3.csv')
X_train, X_test = df[df["price_type"] == 0].drop(columns=["target"]), df[df["price_type"] != 0].drop(columns=["target"])
Y_train, Y_test = df[df["price_type"] == 0]["target"], df[df["price_type"] != 0]["target"]
scaler = StandardScaler()

X_train.iloc[:,:] = scaler.fit_transform(X_train.iloc[:,:])
X_test.iloc[:,:] = scaler.transform(X_test.iloc[:,:])

# scaler2 = StandardScaler()
# Y_train = scaler2.fit_transform(Y_train.values.reshape(-1,1))
# Y_test = scaler2.transform(Y_test.values.reshape(-1,1))

Y_train

0         139937.500000
1          60410.714286
2          45164.761264
3          28805.263158
4          13222.591362
              ...      
275294     25450.669915
275295     43525.000000
275296     28727.272727
275297     79068.245559
275298     41160.714286
Name: target, Length: 275299, dtype: float64

In [7]:
#Catboost trying
import typing
import numpy as np
import catboost as cb
train_dataset = cb.Pool(X_train.values, Y_train)
test_dataset = cb.Pool(X_test.values, Y_test)


class CatBoostEvalMetricRaifhack(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        # the larger metric value the better
        return False



    def deviation_metric_one_sample(self, y_pred: typing.Union[float, int], y_true: typing.Union[float, int]) -> float:
        """
        Реализация кастомной метрики для хакатона.

        :param y_true: float, реальная цена
        :param y_pred: float, предсказанная цена
        :return: float, значение метрики
        """
        THRESHOLD = 0.15
        NEGATIVE_WEIGHT = 1.1
        deviation = (y_pred - y_true) / np.maximum(1e-8, y_true)
        if np.abs(deviation) <= THRESHOLD:
            return 0
        elif deviation <= - 4 * THRESHOLD:
            return 9 * NEGATIVE_WEIGHT
        elif deviation < -THRESHOLD:
            return NEGATIVE_WEIGHT * ((deviation / THRESHOLD) + 1) ** 2
        elif deviation < 4 * THRESHOLD:
            return ((deviation / THRESHOLD) - 1) ** 2
        else:
            return 9


    def evaluate(self, y_pred, y_true, weight):
        y_pred = y_pred[0]
        return np.array([self.deviation_metric_one_sample(y_pred[n], y_true[n]) for n in range(len(y_pred))]).mean(), 0


In [10]:
model = cb.CatBoostRegressor(
    iterations=1000,
    learning_rate=0.5,
    eval_metric=CatBoostEvalMetricRaifhack(),
    # task_type="GPU",
    # devices='0:1'
)
model.fit(
    X_train,
    Y_train,
    verbose=True,
    plot=True
    )

C:\Users\gto_n\.conda\envs\raifhack_2021\lib\site-packages\catboost\core.py:1913: UserWarning: Failed to import numba for optimizing custom metrics and objectives
  _check_train_params(params)
Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 5.6258041	total: 1.16s	remaining: 19m 20s
1:	learn: 4.9781718	total: 2.24s	remaining: 18m 36s
2:	learn: 4.6960028	total: 3.29s	remaining: 18m 14s
3:	learn: 4.4780885	total: 4.36s	remaining: 18m 5s
4:	learn: 4.4290860	total: 5.41s	remaining: 17m 57s
5:	learn: 4.3547734	total: 6.48s	remaining: 17m 53s
6:	learn: 4.2906681	total: 7.59s	remaining: 17m 57s
7:	learn: 4.2667761	total: 8.66s	remaining: 17m 54s
8:	learn: 4.2073346	total: 9.74s	remaining: 17m 52s
9:	learn: 4.1959032	total: 10.8s	remaining: 17m 49s
10:	learn: 4.1755002	total: 11.9s	remaining: 17m 48s
11:	learn: 4.1429670	total: 13s	remaining: 17m 48s
12:	learn: 4.1401757	total: 14.1s	remaining: 17m 48s
13:	learn: 4.1086665	total: 15.1s	remaining: 17m 46s
14:	learn: 4.1070084	total: 16.2s	remaining: 17m 44s
15:	learn: 4.0966570	total: 17.3s	remaining: 17m 43s
16:	learn: 4.0814559	total: 18.4s	remaining: 17m 41s
17:	learn: 4.0795567	total: 19.5s	remaining: 17m 42s
18:	learn: 4.0541128	total: 20.6s	remaining: 17m 41s
19:	le

In [16]:
aa = model.predict(
    X_test
)
print(deviation_metric(Y_test.values, aa))

3.4194666075844258


In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge

model_rf = RandomForestRegressor(max_depth=15, n_estimators=150)
model_rf1 = RandomForestRegressor(max_depth=20, n_estimators=150)
model_rf2 = RandomForestRegressor(max_depth=24, n_estimators=180)


models = [model_rf]
scores = {}
for model in models:
    print(model)
    model.fit(X_train.values, Y_train.values)
    tr_pred = model.predict(X_test.values)
    print(deviation_metric(Y_test.values, tr_pred))

RandomForestRegressor(max_depth=15, n_estimators=150)


In [18]:
import torch.utils.data as data_utils
train = data_utils.TensorDataset(torch.tensor(X_train.values).to(device), torch.tensor(Y_train.values).to(device))
train_loader = data_utils.DataLoader(train, batch_size=128, shuffle=True)
test = data_utils.TensorDataset(torch.tensor(X_test.values).to(device), torch.tensor(Y_test.values).to(device))
test_loader = data_utils.DataLoader(test, batch_size=128, shuffle=True)

In [19]:
for features, targets in train_loader:
    num_features = len(features[0])
    num_targets = 1
    break

class FC_net(nn.Module):
    def __init__(self, num_features, num_targets):
        super(FC_net, self).__init__()
        self.fc0 = nn.Linear(num_features, 1024)
        self.bn0 = nn.BatchNorm1d(1024)
        self.do0 = nn.Dropout()
        self.fc1 = nn.Linear(1024, 528)
        self.bn1 = nn.BatchNorm1d(528)
        self.do1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(528, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.do2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.do3 = nn.Dropout(p=0.2)
        self.fc4 = nn.Linear(128, num_targets)

    def forward(self, x):
        x = self.fc0(x)
        x = self.bn0(x)
        x = F.elu(x)
        # x = self.do0(x)

        x = self.fc1(x)
        x = self.bn1(x)
        x = F.elu(x)
        # x = self.do1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = F.elu(x)
        # x = self.do2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = F.elu(x)
        # x = self.do3(x)

        x = F.elu(self.fc4(x))
        return x

name = 'Linear_5layers_fc_bn_elu.pth'
filename = './VAE/models_and_weights/' + name
net = FC_net(num_features=num_features, num_targets=num_targets).to(device).double()

In [20]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        eps = 10e-8
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y) + eps)
        return loss

In [22]:
lr = 1e-8
optimizer = optim.Adam(net.parameters(), lr=lr)
print(summary(net), (1, num_features))
# lr = 0.01024
running_mape = 0
now = datetime.datetime.now()
tensorboard_path = './tensorboard/' + name[:-4] + '_' + str(now).replace(' ', '_').replace(':', '-')[:-7]
writer = SummaryWriter(tensorboard_path)
running_train_loss = 0.0
running_test_loss = 0.0
minimal_loss = 1e15
previous_loss = 'None'
batch = 0
print('The train is starts!')
batch = 0
rmse = RMSELoss()
while lr > 1e-15:
    for features, targets in train_loader:
        net.zero_grad()
        y_predict = net(features)
        loss = rmse(torch.unsqueeze(targets, 1), y_predict)
        loss.backward()
        optimizer.step()
        batch += 1
        running_train_loss += loss.cpu().detach().numpy()  # batch size
        if ((batch + 1) % 1000) == 0:
            with torch.no_grad():
                test_loader = data_utils.DataLoader(test, batch_size=128, shuffle=True)
                for features, targets in test_loader:
                    net.eval()
                    y_predict = net(features)
                    raifhack = deviation_metric(targets.cpu().detach().numpy(), y_predict.cpu().detach().numpy())
                    break
                net.train()
            running_train_loss /= 1000
            writer.add_scalars(f'loss', {
                'training': running_train_loss,
            }, batch + 1)
            writer.add_scalars(f'raifhack', {
                'raif_train': raifhack,
            }, batch + 1)
            if previous_loss == 'None':
                previous_loss = running_train_loss
                print('previous loss is changed to ', previous_loss)
            if (running_train_loss < minimal_loss):
                minimal_loss = running_train_loss
                file = open('./Data/min_vae_loss.txt', 'w')
                file.write(str(minimal_loss))
                file.close()
                torch.save({
                    'epoch': batch,
                    'model_state_dict': net.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': minimal_loss,
                    'raifhack': raifhack,
                }, 'Data/Models_and_weights_' + name)
                print('NEW WEIGHTS ARE SAVED! Min_loss = ' + str(minimal_loss))
                print('Raif_metrics =', raifhack)
            running_train_loss = 0

    # for features, targets in test_loader:
    #     net.eval()
    #     with torch.no_grad():
    #         y_predict = net(features)
    #         loss = deviation_metric(targets, y_predict)
    # running_test_loss += loss.cpu().detach().numpy()
    #
    # batch += 1
    # if ((batch + 1) % 10) == 0:
    #     running_train_loss /= 50
    #     running_test_loss /= 50
    #     writer.add_scalars(f'loss', {
    #         'training': running_train_loss,
    #         'test': running_test_loss
    #     }, batch + 1)
    #
    #     if previous_loss == 'None':
    #         previous_loss = running_train_loss
    #         print('previous loss is changed to ', previous_loss)
    #     if (running_train_loss < minimal_loss):
    #         minimal_loss = running_train_loss
    #         file = open('./Data/min_vae_loss.txt', 'w')
    #         file.write(str(minimal_loss))
    #         file.close()
    #         torch.save({
    #             'epoch': batch,
    #             'model_state_dict': net.state_dict(),
    #             'optimizer_state_dict': optimizer.state_dict(),
    #             'loss': minimal_loss
    #         }, './models_and_weights/general_for_' + name)
    #         print('NEW WEIGHTS ARE SAVED! Min_loss = ' + str(minimal_loss))
    #     # Сохранение наиболее удачных весов (по тестовой выборке)
    #     if ((batch + 1) % 500) == 0:
    #         if (abs(previous_loss - running_train_loss) / previous_loss) < 0.1:
    #             lr = lr / 2
    #             optimizer.param_groups[0]['lr'] = lr
    #             print('new lr = ', lr, ' Batch= ', batch + 1)
    #         previous_loss = running_train_loss
    #     running_train_loss = 0
    #     running_test_loss = 0

Layer (type:depth-idx)                   Param #
FC_net                                   --
├─Linear: 1-1                            40,960
├─BatchNorm1d: 1-2                       2,048
├─Dropout: 1-3                           --
├─Linear: 1-4                            541,200
├─BatchNorm1d: 1-5                       1,056
├─Dropout: 1-6                           --
├─Linear: 1-7                            135,424
├─BatchNorm1d: 1-8                       512
├─Dropout: 1-9                           --
├─Linear: 1-10                           32,896
├─BatchNorm1d: 1-11                      256
├─Dropout: 1-12                          --
├─Linear: 1-13                           129
Total params: 754,481
Trainable params: 754,481
Non-trainable params: 0
================================================================= (1, 39)
The train is starts!
previous loss is changed to  198948.64978231266
NEW WEIGHTS ARE SAVED! Min_loss = 198948.64978231266
Raif_metrics = 9.900000000000002
NEW WEI

KeyboardInterrupt: 